In [16]:
import pandas as pd
%pip install openpyxl

import openpyxl
df = pd.read_excel("shbm_poetry.xlsx")
df.head()

Note: you may need to restart the kernel to use updated packages.


,Text content,Title
0,حبيـبتي حالـي من الشــوق دانــي<br />,لا تسـأل مجـرّب
1,أشــرّق بحـبّــك .. واعـوّد أغـرّب<br />,NaN
2,<br />,NaN
3,أنـانـي فْـ وصـلك لـكـن يالأنـاني<br />,NaN
4,تـرى الأنانـيّـه فْـ طـبـعـك تـخــرّب<br />,NaN


In [17]:
import re

def strip_html_tags(text):
    """Replace common HTML tags with appropriate whitespace."""
    if pd.isna(text):
        return text
    text = str(text)
    # Replace <br>, <br/>, <br /> with newline
    text = re.sub(r'<br\s*/?>', '\n', text, flags=re.IGNORECASE)
    # Remove other common tags (tr, td, etc.) with a space
    text = re.sub(r'<[^>]+>', ' ', text)
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply to Text content and any other text columns
df['Text content'] = df['Text content'].apply(strip_html_tags)
# Optionally apply to Title if needed
# df['Title'] = df['Title'].apply(strip_html_tags)

# Verify
df.head()

,Text content,Title
0,حبيـبتي حالـي من الشــوق دانــي,لا تسـأل مجـرّب
1,أشــرّق بحـبّــك .. واعـوّد أغـرّب,NaN
2,,NaN
3,أنـانـي فْـ وصـلك لـكـن يالأنـاني,NaN
4,تـرى الأنانـيّـه فْـ طـبـعـك تـخــرّب,NaN


In [19]:
# Forward-fill Title to duplicate down until next non-empty Title
df['Title'] = df['Title'].replace('', pd.NA).ffill()

# Verify
print(f"Rows before removine NA: {len(df)}")
df.head(100)

Rows before removine NA: 11773


,Text content,Title
0,حبيـبتي حالـي من الشــوق دانــي,لا تسـأل مجـرّب
1,أشــرّق بحـبّــك .. واعـوّد أغـرّب,لا تسـأل مجـرّب
2,,لا تسـأل مجـرّب
3,أنـانـي فْـ وصـلك لـكـن يالأنـاني,لا تسـأل مجـرّب
4,تـرى الأنانـيّـه فْـ طـبـعـك تـخــرّب,لا تسـأل مجـرّب
...,...,...
95,هي قصيدة شاعرٍ وضّح لك اللي فـي ظميره,هواء الإمارات وثراها
96,إيْـعَـرف وش علّـته لكن يجـهل وشـ دواها,هواء الإمارات وثراها
97,,هواء الإمارات وثراها
98,غاص بالهوجاس باعماق المحيطات الغزيره,هواء الإمارات وثراها


In [20]:
# Remove rows where Text content is NaN or empty
df = df.dropna(subset=['Text content'])
df = df[df['Text content'].str.strip() != '']

# Reset index if needed
df = df.reset_index(drop=True)

# Verify

print(f"Rows after cleaning: {len(df)}")
df.head()

Rows after cleaning: 8447


,Text content,Title
0,حبيـبتي حالـي من الشــوق دانــي,لا تسـأل مجـرّب
1,أشــرّق بحـبّــك .. واعـوّد أغـرّب,لا تسـأل مجـرّب
2,أنـانـي فْـ وصـلك لـكـن يالأنـاني,لا تسـأل مجـرّب
3,تـرى الأنانـيّـه فْـ طـبـعـك تـخــرّب,لا تسـأل مجـرّب
4,لحظات حبّـك حطّـمت لي كـياني,لا تسـأل مجـرّب


In [31]:
def has_non_arabic_chars(text):
    return bool(re.search(r'[A-Za-z0-9]', str(text)))

# Flag invalid in either Title or Text content
df['invalid'] = df.apply(lambda row: has_non_arabic_chars(row['Text content']) or has_non_arabic_chars(row['Title']), axis=1)

# Show problematic rows with row index, Title, and Text content
problem_lines = df[df['invalid']][['Title', 'Text content']]
problem_lines = problem_lines.reset_index()  # keeps the original row number in a column named 'index'

print("Problematic lines (English letters or numbers) with row number:")
print(problem_lines)
print("Problematic lines (English letters or numbers) in Title or Text content:")
print(problem_lines)
problem_lines.to_csv("current_poems_flaud.csv", index=False)

Problematic lines (English letters or numbers) with row number:
     index                Title  \
0      946         واحد x إثنين   
1      947         واحد x إثنين   
2      948         واحد x إثنين   
3      949         واحد x إثنين   
4      950         واحد x إثنين   
..     ...                  ...   
203   8322    ذكرياتي مَعَ راشد   
204   8348            سؤالك غلط   
205   8364            سؤالك غلط   
206   8365            سؤالك غلط   
207   8428  أميرة قلوب الأنقياء   

                                          Text content  
0                       كل من ألاقي نور في وسط الاحلام  
1                       جتني ورود الشوق تبكي على النوم  
2                     ما نمت من عقب اللـقا عـشرة ايّام  
3                          كل من أغض العين أذكره وآقوم  
4                  الله بـلاني في المحـبّـه عـلى تـوام  
..                                                 ...  
203                  حتـى في وجـه رجّـالٍ&nbsp; كـفيـف  
204        أحسبـه حقيقـي بل حسبتـه مجــرّد زيــف&nbsp;  
20

In [35]:
# Count lines per poem
lines_per_poem = df.groupby('Title').size().reset_index(name='line_count')

# Flag poems with odd number of lines
lines_per_poem['odd_lines'] = lines_per_poem['line_count'] % 2 != 0

# Get only poems with odd line counts
odd_poems = lines_per_poem[lines_per_poem['odd_lines']]

print(f"Poems with odd number of lines (potential garbage lines): {len(odd_poems)}")
print(odd_poems)
odd_poems.to_csv("garbage_lines.csv", index=False)

Poems with odd number of lines (potential garbage lines): 17
                    Title  line_count  odd_lines
35          إفهـم مكـانـك          41       True
42              اسـتجـواب          25       True
75                القـادر          17       True
89                بعـلبـك          21       True
102      تعــال واسـألنـي          25       True
137  خمسين عام قبل و بعـد          81       True
164                  سؤال          19       True
168       سـمح الـجـنــاب          53       True
170            سلام تسليم          43       True
203              عام زايد          47       True
204  عجايب وشنذاره وغيّاث          49       True
285            مرثية زايد          21       True
304       ميـلاد أبوخالـد           9       True
313           هدهد سليمان         121       True
321                   هند          13       True
335             وقفة عرفة          39       True
338               يا أحمد          19       True
